# Widget Development Notebook

This notebook is for developing and testing the pose estimation and video widgets.
It loads NWB files from DANDI and displays the widgets with hot-reload enabled.

In [1]:
# Enable anywidget Hot Module Replacement (HMR) for development
# Changes to .js files will automatically update the widget without kernel restart
import os
os.environ["ANYWIDGET_HMR"] = "1"

# Enable matplotlib inline for any plots
%matplotlib inline

In [2]:
import h5py
import remfile
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient

# Connect to DANDI and get the dandiset
dandiset_id = "000409"
client = DandiAPIClient()
dandiset = client.get_dandiset(dandiset_id, "draft")

# =============================================================================
# Session EIDs for NEW format files (desc-raw / desc-processed)
# =============================================================================

# Complete session with 2 probes - Session 1 (NYU-39, 2021-05-10, angelakilab)
# - Full data: all videos, pose estimation, spike sorting for both probes
TWO_PROBE_SESSION_EID_1 = "6ed57216-498d-48a6-b48b-a243a34710ea"

# Complete session with 2 probes - Session 2 (NYU-39, 2021-05-11, angelakilab)
# - Full data: all videos, pose estimation, spike sorting for both probes
TWO_PROBE_SESSION_EID_2 = "35ed605c-1a1a-47b1-86ff-2b56144f55af"

# Complete session with 1 probe (NYU-46, 2021-06-25, angelakilab)  
# - Full data: all videos, pose estimation, spike sorting for probe01
ONE_PROBE_SESSION_EID = "64e3fb86-928c-4079-865c-b364205b502e"

# Choose which session to use
session_eid = TWO_PROBE_SESSION_EID_1  # Change to TWO_PROBE_SESSION_EID_2 or ONE_PROBE_SESSION_EID

# =============================================================================
# Fetch assets by EID
# =============================================================================

# First, filter assets by EID
session_assets = [asset for asset in dandiset.get_assets() if session_eid in asset.path]

# Then, extract raw and processed files
raw_asset = next((asset for asset in session_assets if "desc-raw" in asset.path), None)
processed_asset = next((asset for asset in session_assets if "desc-processed" in asset.path), None)

print(f"Session EID: {session_eid}")
print(f"\nRaw file:       {raw_asset.path if raw_asset else 'Not found'}")
print(f"Processed file: {processed_asset.path if processed_asset else 'Not found'}")

Session EID: 6ed57216-498d-48a6-b48b-a243a34710ea

Raw file:       sub-NYU-39/sub-NYU-39_ses-6ed57216-498d-48a6-b48b-a243a34710ea_desc-raw_ecephys.nwb
Processed file: sub-NYU-39/sub-NYU-39_ses-6ed57216-498d-48a6-b48b-a243a34710ea_desc-processed_behavior+ecephys.nwb


In [3]:
# Load NWB files via streaming
raw_url = raw_asset.get_content_url(follow_redirects=1, strip_query=True)
processed_url = processed_asset.get_content_url(follow_redirects=1, strip_query=True)

# Wrap remfile with h5py.File for pynwb compatibility
raw_h5 = h5py.File(remfile.File(raw_url), mode="r")
processed_h5 = h5py.File(remfile.File(processed_url), mode="r")

io_raw = NWBHDF5IO(file=raw_h5, mode="r", load_namespaces=True)
io_processed = NWBHDF5IO(file=processed_h5, mode="r", load_namespaces=True)

nwbfile_raw = io_raw.read()
nwbfile_processed = io_processed.read()

print(f"Raw NWB loaded: {nwbfile_raw.identifier}")
print(f"Processed NWB loaded: {nwbfile_processed.identifier}")

Raw NWB loaded: 902d6b43-4a62-4f1d-8c23-21b88edbd35c
Processed NWB loaded: cf2e6fd7-31e6-4860-a09b-8c4eae2ab010


In [4]:
# Get video URLs from the raw NWB file
from nwb_video_widgets import NWBFileVideoPlayer

video_s3_urls = NWBFileVideoPlayer.get_video_urls_from_dandi(nwbfile_raw, raw_asset)
print("Available videos:")
for name, url in video_s3_urls.items():
    print(f"  {name}: {url[:80]}...")

/home/heberto/development/nwb-video-widgets/.venv/lib/python3.12/site-packages/anywidget/_util.py:283: UserWarning: anywidget: Live-reloading feature is disabled. To enable, please install the 'watchfiles' package.
  start_thread=_should_start_thread(path),


Available videos:
  VideoBodyCamera: https://dandiarchive.s3.amazonaws.com/blobs/7f8/b5c/7f8b5c2a-3392-412e-8434-94d2...
  VideoLeftCamera: https://dandiarchive.s3.amazonaws.com/blobs/267/135/267135c8-546a-49cf-bba9-6903...
  VideoRightCamera: https://dandiarchive.s3.amazonaws.com/blobs/ac6/137/ac613719-2106-4a89-b134-0583...


## Multi-Camera Video Player

In [5]:
NWBFileVideoPlayer(nwbfile_raw, raw_asset)

## Pose Estimation Widget

In [6]:
from nwb_video_widgets import NWBPoseEstimationWidget

NWBPoseEstimationWidget(
    nwbfile=nwbfile_processed,
    video_urls=video_s3_urls,
    camera_to_video_key={
        "LeftCamera": "VideoLeftCamera",
        "BodyCamera": "VideoBodyCamera",
        "RightCamera": "VideoRightCamera",
    }
)